In [3]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.0 MB/s eta 0:00:00 MB/s eta 0:00:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 29.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [70]:
from netCDF4 import Dataset

url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_2014.nc.ascii?wind_speed[90:1:90][156:1:156][1101:1:1101]"
ds = Dataset(url)                     # opens the remote dataset
print(ds.variables.keys())  # list available variables


#data = ds.variables['lat'][:]        # read the entire var1 array
#time = ds.variables['wind_speed']      # read the first 10 time points

OSError: [Errno -90] NetCDF: file not found: 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_2025.nc.ascii?wind_speed[90:1:90][156:1:156][1101:1:1101]'

In [52]:
ws

<class 'netCDF4.Variable'>
int16 wind_speed(day, lat, lon)
    _FillValue: 32767
    units: m/s
    description: Daily Mean Wind Speed (10m)
    long_name: vs
    standard_name: vs
    missing_value: 32767
    dimensions: lon lat time
    grid_mapping: crs
    coordinate_system: WGS84,EPSG:4326
    scale_factor: 0.1
    add_offset: 0.0
    coordinates: lon lat time
    _Unsigned: true
    _ChunkSizes: [ 27 147 347]
unlimited dimensions: 
current shape = (1, 1, 1)
filling off

In [53]:

url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_2025.nc"
ds = Dataset(url)

# Get latitude and longitude arrays
latitudes = ds.variables['lat'][:]  # Shape: (585,)
longitudes = ds.variables['lon'][:]  # Shape: (1386,)

# Get coordinates at specified indices
lat_value = latitudes[107]
lon_value = longitudes[275]

print(f"Latitude at index 107: {lat_value}")
print(f"Longitude at index 275: {lon_value}")

Latitude at index 107: 44.94166666666667
Longitude at index 275: -113.30833330000002


In [49]:
{dim: len(ds.dimensions[dim]) for dim in ds.dimensions}

{'day': 1, 'lat': 1, 'lon': 1}

In [50]:
ws = ds.variables['wind_speed']
raw = ws[:]
print(raw)

[[[3.7]]]


In [14]:
import numpy as np
raw = np.ma.masked_equal(raw, ws._FillValue)

# 5) Apply scale and offset to get real wind‑speed in m/s:
wind_speed_mps = raw * ws.scale_factor + ws.add_offset

print(wind_speed_mps)

[[[--]]]


In [11]:
import numpy as np
from netCDF4 import Dataset, num2date
from datetime import datetime
from netCDF4 import Dataset


# 1. Open dataset — replace with local path or OPeNDAP URL
url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_2014.nc"  # or "your_file.nc"
ds = Dataset(url)

# 2. Read coordinate variables
lats = ds.variables['lat'][:]     # shape: (n_lat,)
lons = ds.variables['lon'][:]     # shape: (n_lon,)
times = ds.variables['day'][:]    # shape: (n_time,)
time_units = ds.variables['day'].units

# 3. Convert real-world coordinates to nearest index
target_lat = 41.87
target_lon = -87.62

lat_idx = np.abs(lats - target_lat).argmin()
lon_idx = np.abs(lons - target_lon).argmin()

print(lat_idx,lon_idx)

# Example valid latitude and longitude ranges for GRIDMET (for US data)
lat_min, lat_max = 24.396308, 49.384358  # Approximate latitude range
lon_min, lon_max = -125.0, -66.93457     # Approximate longitude range


# Check if the target coordinates are within the valid bounds
if lat_min <= target_lat <= lat_max and lon_min <= target_lon <= lon_max:
    print("Coordinates are within the valid range.")
else:
    print("Coordinates are out of the valid range.")

# 4. Convert date to index
target_date = datetime(2024, 1, 5)  # Example: April 1, 2023
converted_times = num2date(times, units=time_units)
day_idx = np.argmin(np.abs(np.array(converted_times) - target_date))

print(day_idx)
# 5. Retrieve wind speed
wind_var = ds.variables['wind_speed']
raw_value = wind_var[day_idx, lat_idx, lon_idx]

# 6. Apply scale and missing value mask
scale_factor = wind_var.scale_factor if hasattr(wind_var, 'scale_factor') else 1
fill_value = wind_var._FillValue if hasattr(wind_var, '_FillValue') else None

# Convert raw value
# if raw_value == fill_value:
#     wind_speed = None
# else:
#     wind_speed = raw_value * scale_factor

# 7. Print results
print(f"Date: {converted_times[day_idx]}")
print(f"Location: lat={lats[lat_idx]:.4f}, lon={lons[lon_idx]:.4f}")
print(f"Wind speed: {raw_value} m/s")

181 892
Coordinates are within the valid range.
364
Date: 2014-12-31 00:00:00
Location: lat=41.8583, lon=-87.6000
Wind speed: 7.4 m/s


In [127]:
url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmn/tmmn_2013.nc"
ds = Dataset(url)

In [128]:
temp_var = ds.variables['air_temperature']
raw_value = temp_var[day_idx, lat_idx, lon_idx]
print(raw_value)
print(temp_var)

280.8
<class 'netCDF4.Variable'>
int16 air_temperature(day, lat, lon)
    _FillValue: 32767
    units: K
    description: Daily Minimum Temperature
    long_name: tmmn
    standard_name: tmmn
    missing_value: 32767
    dimensions: lon lat time
    grid_mapping: crs
    coordinate_system: WGS84,EPSG:4326
    scale_factor: 0.1
    add_offset: 210.0
    coordinates: lon lat
    _Unsigned: true
    _ChunkSizes: [ 61  98 231]
unlimited dimensions: 
current shape = (365, 585, 1386)
filling off


In [129]:
import numpy as np

fill_value = 32767

def decode_temp(raw):
    if raw == fill_value:
        return np.nan  # or None
    return (raw - 273.15) * (9/5) + 32

In [130]:
decode_temp(raw_value)

np.float64(45.77000000000006)

In [131]:
url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_2013.nc"
ds = Dataset(url)
temp_var = ds.variables['air_temperature']
raw_value = temp_var[day_idx, lat_idx, lon_idx]
print(raw_value)
decode_temp(raw_value)

288.4


np.float64(59.45)

In [135]:
url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_2013.nc"
ds = Dataset(url)
temp_var = ds.variables['burning_index_g']
raw_value = temp_var[day_idx, lat_idx, lon_idx]
print(raw_value)

0.0


In [142]:
url = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_2013.nc"
ds = Dataset(url)
sr = ds.variables['surface_downwelling_shortwave_flux_in_air']
raw_value = sr[day_idx, lat_idx, lon_idx]
print(raw_value)

276.3


In [133]:
print(len(lons))

1386


In [134]:
print(raw_value,lon_idx,lat_idx,day_idx)

0.0 148 307 231


In [67]:
wind_var[90,156,1101]

np.float64(3.6)

In [21]:
import numpy as np
import pandas as pd
import requests
from netCDF4 import Dataset, num2date
from datetime import datetime

FEATURES = [
    'pr', 'rmax', 'rmin', 'sph', 'srad',
    'tmmn', 'tmmx', 'vs', 'bi',
    'fm100', 'fm1000', 'erc', 'etr', 'pet', 'vpd'
]

FEATURE_VAR_MAP = {
    "pr":    "precipitation_amount",
    "rmax":  "relative_humidity",
    "rmin":  "relative_humidity",
    "sph":   "specific_humidity",
    "srad":  "surface_downwelling_shortwave_flux_in_air",
    "tmmn":  "air_temperature",
    "tmmx":  "air_temperature",
    "vs":    "wind_speed",
    "bi":    "burning_index_g",
    "fm100": "dead_fuel_moisture_100hr",
    "fm1000":"dead_fuel_moisture_1000hr",
    "erc":   "energy_release_component-g",
    "etr":   "potential_evapotranspiration",
    "pet":   "potential_evapotranspiration",
    "vpd":   "mean_vapor_pressure_deficit"
}

BASE_URL = (
    "http://thredds.northwestknowledge.net:8080/"
    "thredds/dodsC/MET/{var}/{var}_{year}.nc"
)

def fetch_75_ascii(lat, lon, date_str):
    # Extract year from date string for URL formatting
    year = date_str.split('-')[0]
    
    # 1) Open a tiny var for metadata
    ds0 = Dataset(BASE_URL.format(var='vs', year=year))
    
    # 2. Read coordinate variables
    lats = ds0.variables['lat'][:]     # shape: (n_lat,)
    lons = ds0.variables['lon'][:]     # shape: (n_lon,)
    times = ds0.variables['day'][:]    # shape: (n_time,)
    time_units = ds0.variables['day'].units
    
    # 3. Find indices for lat, lon
    lat_idx = np.abs(lats - lat).argmin()
    lon_idx = np.abs(lons - lon).argmin()
    
    # 4. Convert time to datetime and find index
    dates = num2date(times, time_units)
    target = datetime.strptime(date_str, "%Y-%m-%d")
    
    # Compare year, month, and day directly instead of using date() method
    date_matches = [
        (d.year == target.year and 
         d.month == target.month and 
         d.day == target.day) 
        for d in dates
    ]
    time_idx = int(np.where(date_matches)[0][0])
    
    # 5) Define 75-day window
    start, end = time_idx - 60, time_idx + 14
    
    # 6) Inspect variable dimensions for ordering
    print(ds0)
    var_dims = ds0.variables['wind_speed'].dimensions  # e.g. ('time','lat','lon')
    
    # Get dimension names
    time_dim = [d for d in var_dims if 'time' in d.lower() or 'day' in d.lower()][0]
    lat_dim = [d for d in var_dims if 'lat' in d.lower()][0]
    lon_dim = [d for d in var_dims if 'lon' in d.lower()][0]
    
    ds0.close()
    
    # 7) Fetch all features via ASCII subset
    data = {}
    for var in FEATURES:
        # build slice strings in the same order as var_dims
        slices = []
        for d in var_dims:
            if d == lat_dim:   
                slices.append(f"[{lat_idx}:1:{lat_idx}]")
            elif d == lon_dim: 
                slices.append(f"[{lon_idx}:1:{lon_idx}]")
            elif d == time_dim: 
                slices.append(f"[{start}:1:{end}]")
            else:
                raise RuntimeError(f"Unexpected dim {d} in {var}")
        
        url = (
            BASE_URL.format(var=var, year=year)
            + ".ascii?" + FEATURE_VAR_MAP[var] + "".join(slices)
        )
        print(url)
        resp = requests.get(url)
        
        print(resp,resp.text.strip())
        # vals = [float(x) for x in resp.text.strip().split()]
        # if len(vals) != 75:
        #     raise RuntimeError(f"{var}: got {len(vals)} points, expected 75")
        # data[var] = vals
        break
    
    # 8) Build a DataFrame with a 75-day DatetimeIndex
    idx = pd.date_range(end=date_str, periods=75)
    return pd.DataFrame(data, index=idx)

# ── Example ────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    df75 = fetch_75_ascii(
        lat=27.591585,
        lon=-82.069418,
        date_str="2014-05-10"
    )
    print(df75.shape)  # → (75, 15)
    print(df75.head())

<class 'netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    geospatial_bounds_crs: EPSG:4326
    Conventions: CF-1.6
    geospatial_bounds: POLYGON((-124.7666666333333 49.400000000000000, -124.7666666333333 25.066666666666666, -67.058333300000015 25.066666666666666, -67.058333300000015 49.400000000000000, -124.7666666333333 49.400000000000000))
    geospatial_lat_min: 25.066666666666666
    geospatial_lat_max: 49.40000000000000
    geospatial_lon_min: -124.7666666333333
    geospatial_lon_max: -67.058333300000015
    geospatial_lon_resolution: 0.041666666666666
    geospatial_lat_resolution: 0.041666666666666
    geospatial_lat_units: decimal_degrees north
    geospatial_lon_units: decimal_degrees east
    coordinate_system: EPSG:4326
    author: John Abatzoglou - University of Idaho, jabatzoglou@uidaho.edu
    date: 04 July 2019
    note1: The projection information for this file is: GCS WGS 1984.
    note2: Citation: Abatzoglou, J.T., 2013, Development of